# Generating sequential synthetic samples

#### Importing the required packages

In [5]:
import os 
import json
import pickle as pkl
import pandas as pd

from ydata.connectors import GCSConnector
from ydata.connectors import LocalConnector
from ydata.utils.formats import read_json
from ydata.connectors.filetype import FileType

from ydata.synthesizers.timeseries import TimeSeriesSynthesizer
from ydata.metadata import Metadata
from ydata.dataset import Dataset
from ydata.utils.data_types import DataType

In [6]:
def generate(nsamples: int):
    synth = TimeSeriesSynthesizer.load('outputs/model.pkl')
    return synth.sample(n_samples=n_samples)

#### Loading the artifacts from previous steps

In [7]:
#Check wether the synth is already trained
with open('outputs/shape.json', 'r') as shape:
    real_shape = json.load(shape)

In [ ]:
try:
    n_samples=os.environ['NSAMPLE']
except KeyError:
    n_samples=real_shape[0][1]
    
try:
    path=os.environ['OUTPUT_PATH']
except KeyError:
    path='gs://ydata_testdata/timeseries/berka/synth_samples.csv'

## Generating synthetic samples

In [ ]:
synth_samples = generate(nsamples=n_samples)

In [ ]:
# Initialize the local connector
connector = LocalConnector()
connector.write_file(data= synth_samples, path='outputs/synth_data.csv', index=False)

### Saving generated samples to a remote storage

#### Init the connector to the remote storage

In [ ]:
token = read_json('credentials/gcs_credentials.json')
read_connector = GCSConnector('ydatasynthetic', keyfile_dict=token)

#### Write to a provided path

In [ ]:
read_connector.write_file(data=synth_samples, path=path, file_type = FileType.CSV)

In [ ]:
metadata = {
    'outputs' : [{
      'type': 'table',
      'storage': 'inline',
      'format': 'csv',
      'header': list(synth_samples.columns),
      'source': str(synth_samples)
    }]
  }

with open("mlpipeline-ui-metadata.json", 'w') as metadata_file:
    json.dump(metadata, metadata_file)